In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# 1. Open the notebook created for Lab-cleaning-numerical-data.

In [2]:
customer_df = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')

In [3]:
cols = []
for colname in customer_df.columns:
    cols.append(colname.lower().replace(' ', '_'))
customer_df.columns = cols

customer_df.rename(columns={'employmentstatus':'employment_status'}, inplace = True)

In [4]:
customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'], errors='coerce')

In [5]:
numerical_df = customer_df.select_dtypes(include = np.number)

In [6]:
def discrete_continuous(df):
    
    discrete_df = pd.DataFrame()
    continuous_df = pd.DataFrame()
    
    for column in df.columns:
        if df[column].nunique() <= 100:
            discrete_df[column] = df[column]
        else:
            continuous_df[column] = df[column]
    
    return discrete_df, continuous_df

In [7]:
discrete_df, continuous_df = discrete_continuous(numerical_df)

In [8]:
def plot(df):
    for column in df.columns:
        if df[column].nunique() <= 100:
            plt.figure(figsize=(20, 6))
            sns.countplot(x=df[column], data=df)
            plt.title(f'Count Plot of {column}')
            plt.xticks(rotation=90)  # Rotate x-axis labels vertically
            plt.show()
        else:
            plt.figure(figsize=(8, 6))
            sns.histplot(x=df[column], bins=20, kde=True)
            plt.title(f'Histogram of {column}')
            plt.xlabel(column)
            plt.ylabel('Frequency')
            plt.show()

In [9]:
continuous_df = continuous_df[(continuous_df['customer_lifetime_value'] <= 65000) & (continuous_df['total_claim_amount'] <= 2500)]
customer_df = customer_df[(customer_df['customer_lifetime_value'] <= 65000) & (customer_df['total_claim_amount'] <= 2500)]
numerical_df = numerical_df[(numerical_df['customer_lifetime_value'] <= 65000) & (numerical_df['total_claim_amount'] <= 2500)]

In [10]:
numerical_df.to_csv('numerical_230821.csv', index=False)

# 2. Find all of the categorical data. Save it in a categorical_df variable.

In [11]:
categorical_df = customer_df.select_dtypes(include = np.object)
categorical_df.drop('customer', axis=1, inplace=True)
categorical_df

C:\Users\PC\AppData\Local\Temp\ipykernel_10536\806269968.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_df = customer_df.select_dtypes(include = np.object)


,state,response,coverage,education,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9131,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
9132,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


# 3. Check for NaN values and decide what to do with them, do it now.

In [12]:
categorical_df.isna().sum()

state                0
response             0
coverage             0
education            0
employment_status    0
gender               0
location_code        0
marital_status       0
policy_type          0
policy               0
renew_offer_type     0
sales_channel        0
vehicle_class        0
vehicle_size         0
dtype: int64

# 4. Check all unique values of columns.

In [13]:
for column in categorical_df.columns:
    unique_values = categorical_df[column].unique()
    print(f"Unique values in '{column}': {unique_values}")

Unique values in 'state': ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
Unique values in 'response': ['No' 'Yes']
Unique values in 'coverage': ['Basic' 'Extended' 'Premium']
Unique values in 'education': ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
Unique values in 'employment_status': ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
Unique values in 'gender': ['F' 'M']
Unique values in 'location_code': ['Suburban' 'Rural' 'Urban']
Unique values in 'marital_status': ['Married' 'Single' 'Divorced']
Unique values in 'policy_type': ['Corporate Auto' 'Personal Auto' 'Special Auto']
Unique values in 'policy': ['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'
 'Corporate L1' 'Personal L2' 'Special L1' 'Special L3']
Unique values in 'renew_offer_type': ['Offer1' 'Offer3' 'Offer2' 'Offer4']
Unique values in 'sales_channel': ['Agent' 'Call Center' 'Web' 'Branch']
Unique values in 'vehicle_class': ['Two-Door Car' 'Four-Door Car' 'S

# 5. Check dtypes. Do they all make sense as categorical data?

In [14]:
for column in categorical_df.columns:
    dtypes = categorical_df[column].dtype
    print(f"Type of '{column}': {dtypes}")

Type of 'state': object
Type of 'response': object
Type of 'coverage': object
Type of 'education': object
Type of 'employment_status': object
Type of 'gender': object
Type of 'location_code': object
Type of 'marital_status': object
Type of 'policy_type': object
Type of 'policy': object
Type of 'renew_offer_type': object
Type of 'sales_channel': object
Type of 'vehicle_class': object
Type of 'vehicle_size': object


Apart from "customer" that I already dropped, yes. "customer" values are unique, so it´s not categorical data.

# 6. Does any column contain alpha and numeric data? Decide how to clean it and do it now.

Yes, but I think it´s right. Nonetheless, we can shorter "Offer1" to "O1".

In [15]:
substitutions = {
    'Offer1': 'O1',
    'Offer2': 'O2',
    'Offer3': 'O3',
    'Offer4': 'O4'
}

categorical_df['renew_offer_type'] = categorical_df['renew_offer_type'].map(substitutions)

categorical_df['renew_offer_type']

0       O1
1       O3
2       O1
3       O1
4       O1
        ..
9129    O2
9130    O1
9131    O1
9132    O3
9133    O4
Name: renew_offer_type, Length: 9126, dtype: object

# 7. Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions and do it now.

In [16]:
categorical_df['employment_status'].value_counts()

Employed         5693
Unemployed       2315
Medical Leave     431
Disabled          405
Retired           282
Name: employment_status, dtype: int64

We could group column employment_status into 'active' or 'inactive'.

In [17]:
substitutions2 = {
    'Employed': 'Active',
    'Unemployed': 'Inactive',
    'Medical Leave': 'Inactive',
    'Disabled': 'Inactive',
    'Retired': 'Inactive'
}

categorical_df['employment_status'] = categorical_df['employment_status'].map(substitutions2)

In [18]:
categorical_df['employment_status'].unique()

array(['Active', 'Inactive'], dtype=object)

# 8. Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?

Unique values in 'policy_type': ['Corporate Auto' 'Personal Auto' 'Special Auto']



Unique values in 'policy': ['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'

In [19]:
categorical_df.head(60)

,state,response,coverage,education,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,Active,F,Suburban,Married,Corporate Auto,Corporate L3,O1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Inactive,F,Suburban,Single,Personal Auto,Personal L3,O3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Active,F,Suburban,Married,Personal Auto,Personal L3,O1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Inactive,M,Suburban,Married,Corporate Auto,Corporate L2,O1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Active,M,Rural,Single,Personal Auto,Personal L1,O1,Agent,Four-Door Car,Medsize
5,Oregon,Yes,Basic,Bachelor,Active,F,Rural,Married,Personal Auto,Personal L3,O2,Web,Two-Door Car,Medsize
6,Oregon,Yes,Basic,College,Active,F,Suburban,Married,Corporate Auto,Corporate L3,O1,Agent,Four-Door Car,Medsize
7,Arizona,No,Premium,Master,Inactive,M,Urban,Single,Corporate Auto,Corporate L3,O1,Agent,Four-Door Car,Medsize
8,Oregon,Yes,Basic,Bachelor,Inactive,M,Suburban,Divorced,Corporate Auto,Corporate L3,O1,Agent,Four-Door Car,Medsize
9,Oregon,No,Extended,College,Active,F,Urban,Married,Special Auto,Special L2,O2,Branch,Four-Door Car,Medsize


We can drop the "policy_type" column because the information is already present in the "policy" column.

# 9. Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.

In [20]:
for column in categorical_df.columns:
    print(f"Unique values in '{column}':")
    unique_values_count = categorical_df[column].value_counts()
    print(unique_values_count)
    print()

Unique values in 'state':
California    3149
Oregon        2599
Arizona       1702
Nevada         880
Washington     796
Name: state, dtype: int64

Unique values in 'response':
No     7818
Yes    1308
Name: response, dtype: int64

Unique values in 'coverage':
Basic       5567
Extended    2737
Premium      822
Name: coverage, dtype: int64

Unique values in 'education':
Bachelor                2744
College                 2681
High School or Below    2618
Master                   741
Doctor                   342
Name: education, dtype: int64

Unique values in 'employment_status':
Active      5693
Inactive    3433
Name: employment_status, dtype: int64

Unique values in 'gender':
F    4657
M    4469
Name: gender, dtype: int64

Unique values in 'location_code':
Suburban    5772
Rural       1772
Urban       1582
Name: location_code, dtype: int64

Unique values in 'marital_status':
Married     5294
Single      2463
Divorced    1369
Name: marital_status, dtype: int64

Unique values in 'policy_

We can group values that don´t represent a high percentage of the data.

In [21]:
substitutions3 = {
    'Bachelor': 'Bachelor',
    'College': 'College',
    'High School or Below': 'High School or Below',
    'Master': 'Higher',
    'Doctor': 'Higher'
}

categorical_df['education'] = categorical_df['education'].map(substitutions3)

In [22]:
categorical_df['education']

0       Bachelor
1       Bachelor
2       Bachelor
3       Bachelor
4       Bachelor
          ...   
9129    Bachelor
9130     College
9131    Bachelor
9132     College
9133     College
Name: education, Length: 9126, dtype: object

In [23]:
substitutions4 = {
    'Four-Door Car': 'Four-Door Car',
    'Two-Door Car': 'Two-Door Car',
    'SUV': 'SUV',
    'Sports Car': 'Sports Car',
    'Luxury SUV': 'Luxury',
    'Luxury Car': 'Luxury'
}
categorical_df['vehicle_class'] = categorical_df['vehicle_class'].map(substitutions4)

In [24]:
categorical_df['vehicle_class']

0        Two-Door Car
1       Four-Door Car
2        Two-Door Car
3                 SUV
4       Four-Door Car
            ...      
9129    Four-Door Car
9130    Four-Door Car
9131    Four-Door Car
9132    Four-Door Car
9133     Two-Door Car
Name: vehicle_class, Length: 9126, dtype: object

In [26]:
# Checking I did everything right
categorical_df.isna().sum()

state                0
response             0
coverage             0
education            0
employment_status    0
gender               0
location_code        0
marital_status       0
policy_type          0
policy               0
renew_offer_type     0
sales_channel        0
vehicle_class        0
vehicle_size         0
dtype: int64

# 10. Save the cleaned catagorical dataframe as categorical.csv You will use this file again this week.

In [27]:
categorical_df.to_csv('categorical_230821.csv', index=False)